# Copyright 2023 Cognite AS

 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at

      <https://www.apache.org/licenses/LICENSE-2.0>

 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.

# Investigating Additional Operational States

We require an investigation into alternative modes of operation of the compressor system. Main variables of interest are the boundary conditions that are used in the simulator model. 

* Inlet pressure (23-PT-92504): 3 barg
* Inlet temperature (23-TT-92504): 37.6 C
* Outlet pressure (transmitters not available, used 23-PT-92536 – 23-PDT-92602 – 0.1 (assumed additional pressure loss)): 12 barg
* Inlet Cooling Pressure (45-PT-93288, measurement not available, assumed) : 7.69 barg
* Inlet Cooling Temperature (45-TT-92606): 16.8 C
* Outlet Cooling Pressure (45-PT-92608): 5.05 barg

 

Controlled variable:

* Gas temp. out of coolers (23-TT-92604A): 34.73 C


## Setup

Retrieving requisite data from CDF. 

In [ ]:
from cognite.client import CogniteClient, ClientConfig
from cognite.client.credentials import OAuthClientCredentials
import os
import pandas as pd

client = CogniteClient(
            ClientConfig(
                client_name="Cognite examples library 0.1.0",
                base_url=os.environ["CDF_URL"],
                project=os.environ["CDF_PROJECT"],
                credentials=OAuthClientCredentials(
                    token_url=os.environ["IDP_TOKEN_URL"],
                    client_id=os.environ["IDP_CLIENT_ID"],
                    # client secret should not be stored in-code, so we load it from an environment variable
                    client_secret=os.environ["IDP_CLIENT_SECRET"],
                    scopes=[os.environ["IDP_SCOPES"]],
                ),
            )
        )



In [ ]:
# Retrieve data from CDF
ts_list = client.time_series.list(limit=-1)
ts_map = {ts.external_id: f"{ts.name}_{ts.unit}_{ts.description}; " for ts in ts_list if not ts.is_string }


In [ ]:
tags = ['23-PT-92504', '23-TIC-92504:Z.X', '23-PT-92536', '23-PDT-92602','45-PT-93288','45-TT-92606','45-PT-92608','23-TT-92604A']

len(tags)

In [ ]:
store={}
for key, val in ts_map.items():
    for tag in tags:
        if tag in val:
            store[key]= val
            continue

In [ ]:
ts_list[0].first()

In [ ]:
start_date = pd.to_datetime('2013-01-01')
end_date = 'now'


df = client.time_series.data.retrieve_dataframe(start=start_date, end=end_date, external_id=list(store.keys()), aggregates=['average'], granularity='1h', include_aggregate_name=False)
df_cdf = df.rename(columns=store)

df_cdf.head()

In [ ]:
# Deriving the outlet pressure
df_cdf['outlet_pressure_bar'] = df_cdf.filter(regex='23-PT-92536')-df_cdf.filter(regex='23-PDT-92602').values - 0.1

In [ ]:
# Drop the not needed ones
cols_to_drop = df_cdf.filter(regex='(23-PT-92536)|(23-PDT-92602)').columns

df_cdf = df_cdf.drop(cols_to_drop,axis=1)

## Long term variable trends

Smoothing the lines and looking at long term trends. 


In [ ]:
import matplotlib.pyplot as plt

# Looking at long term trends

tmp = df_cdf.resample('1D').median()
tmp = tmp/tmp.median()
tmp.plot(figsize=(15,30), subplots=True,grid=True, ylim=(0.5,1.5))



In [ ]:
# Removing obvious outliers with quantiles (also diregarding the noise of the start-up state)

for col in tmp:
    plt.figure()
    tmp_2 = tmp[col].loc[pd.to_datetime('2013-08-01'):]
    filtered = tmp_2[(tmp_2>tmp_2.quantile(0.05))&(tmp_2<tmp_2.quantile(0.95))]
    filtered.plot(figsize=(15,5))
    print(f'{col}: {filtered.std().round(2)}')
    plt.title(col)

### Observations

* H1 2013 period can be dismissed as it has a lot of noise (likely due to start-up of machinery)
* Largest deviations occur as follows:
    * Pressure of CM out (norm. std of 0.11)
    * Outlet pressure (norm. std of 0.08)
    * Inlet pressure (norm. std of 0.04)
    * Temperature In (norm std of 0.03)
* Data for Temperature of gas output starts only from mid 2016; but since it is relatively constant, we can set it to be the same value.
* Step changes in the inlet pressure can be seen which could point to alternative operating regions. 
* Pressure of CM has low effect on simulation result so we can ignore
* Outlet pressure has 2 radically different modes of operation; can check both of those scenarios
* Early 2020 sees pressure output of suction cooler oscillating more clearly (likely due to opening of anti-surge valve)
* Temperature control of Gas Out of suction cooler is more erratic from 2018 onwards.


## Baseline noise level of sensors

Check the baseline noise level of the sensors.

In [ ]:
# Checking the average rolling variance over a 12 hour period
start_date = pd.to_datetime('2014-01-01')
end_date = start_date+ pd.Timedelta(days=1)
df = client.time_series.data.retrieve_dataframe(start=start_date, end=end_date, external_id=list(store.keys()), aggregates=['average'], granularity='1s', include_aggregate_name=False)
df_gran = df.rename(columns=store)


In [ ]:
df_gran.plot(figsize=(15,5))

In [ ]:
# 1 sec granularity over a day of constant operation

df_gran.max()-df_gran.median()

In [ ]:
tmp = df_gran['VAL_23-TIC-92504:Z.X.Value_degC_PH 1stStgSuctCool Gas Out Measured Value; ']

tmp.plot(figsize=(15,5))
tmp.fillna(method='ffill').rolling(3600).median().plot()


In [ ]:
tmp = df_gran['VAL_45-TT-92606:X.Value_None_PH 1stStgDiscClr CoolMed Sply; ']

tmp.plot(figsize=(15,5))
tmp.fillna(method='ffill').rolling(3600).median().plot()


In [ ]:
tmp = df_gran['VAL_23-PT-92504:X.Value_None_PH 1stStgSuctCool Gas Out; ']

tmp.plot(figsize=(15,5))
tmp.fillna(method='ffill').rolling(3600).median().plot()


In [ ]:
# 1 hour granularity over a day of constant operation
(df_gran.resample('1H').mean().max()-df_gran.resample('1H').mean().median())

### Observations

* High variablility encountered for 1 second data (>0.6 bar for pressure sensors and 1.7 C for one temperature sensor)
* 1 hour data has low variability (<0.1 bar for pressure sensors and <0.3 C for temperature)

Looking for operating modes that have greater variability than the threshold found above for 1 hour data

## Identifying different operating modes

Observations of previous

* Inlet pressure has various steady state changes over the historical record (~6). 
* Outlet pressure has ~2-3 noticeable states that should be covered by the regions identified above (aside from single drop identified in mid 2020)
* Potentially can also include a big peak in the inlet temparature (regions of 1.05 and 0.9)

In [ ]:
# Function for outlier removal and s
from scipy.signal import savgol_filter

def outlier_removal_and_smoother(df, col, polyorder=3, window_length=5):
    tmp = df[col]
    
    # remove start-up perido
    tmp = tmp.loc[pd.to_datetime('2013-09-01'):]
    
    # remove outliers
    tmp = tmp[(tmp>tmp.quantile(0.05))&(tmp<tmp.quantile(0.95))]
    
    # SG smoother
    data = tmp.values
    index=tmp.index
    return pd.Series(savgol_filter(data, window_length=window_length, polyorder=polyorder), index=index)

    

In [ ]:
col = df_cdf.filter(regex='23-PT-92504').columns[0]
tmp = outlier_removal_and_smoother(df_cdf, col, window_length=101, polyorder=2)

#df_cdf[col].plot()
tmp.plot(figsize=(15,5))
plt.ylim([2.5,4.0])
plt.ylabel(col)

# break points
index = tmp[(tmp.rolling(24).max()-tmp.rolling(24).mean())>0.02].index
days = set([item.date() for item in index])
for item in days:
    plt.axvline(item,color='k')
    

In [ ]:
intervals = pd.Series(sorted(days))

print(tmp.index[0])
print(intervals[0])

for day in intervals[intervals.diff()>pd.Timedelta(days=14)].values:
    print(day)

### Observations

From the above, we can take a day in the following time period (I'm choosing half way in between):
* 2013-09-01 - 2014-05-30
* 2014-05-30 - 2015-01-02
* 2015-01-02 - 2016-05-29
* 2016-05-28 - 2016-06-23
* 2016-06-23 - 2017-09-18
* 2017-09-18 - 2018-02-06
* 2017-09-15 - 2020-04-07

The later periods are affected by the movement of the anti-surge valve, so these will require closer analysis



## Outlet Pressure Analysis

Checking the values of the outlet pressure

In [ ]:
col = df_cdf.filter(regex='outlet').columns[0]
tmp = outlier_removal_and_smoother(df_cdf, col, window_length=101, polyorder=2)
tmp.plot(figsize=(15,5))
plt.ylabel(col)

index = tmp[(tmp.rolling(24).max()-tmp.rolling(24).mean())>0.1].index
days = set([item.date() for item in index])
for item in days:
    plt.axvline(item,color='k')

## Find all relevant variables

Look at each time period and grab the values

* 2013-09-01 - 2014-05-30
* 2014-05-30 - 2015-01-02
* 2015-01-02 - 2016-05-29
* 2016-05-28 - 2016-06-23
* 2016-06-23 - 2017-09-18
* 2017-09-18 - 2018-02-06
* 2017-09-15 - 2020-04-07

In [ ]:
def plot_and_grab_all_values(df, start, end, ratio=0.5):
    
    
    interval = (end-start)*ratio
    new_start = (start+interval).floor('1h')
    new_end = new_start+pd.Timedelta(hours=1)
    plot_end = new_start+pd.Timedelta(days=1)
    
    print(new_start)
    
    df.loc[new_start:plot_end].plot(subplots=True, figsize=(15,20))
    
    res = df.loc[new_start:new_end].mean().round(2)
    print(res)
    res.name = new_start
    return res
    
    

In [ ]:
store=[]

In [ ]:
res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2013-09-01'), pd.to_datetime('2014-05-30'), ratio=0.3)
store.append(res)

In [ ]:
res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2014-05-31'), pd.to_datetime('2015-01-03'), ratio=0.2)
store.append(res)

In [ ]:
# 2015-01-02 - 2016-05-29

res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2015-01-02'), pd.to_datetime('2016-05-29'), ratio=0.2)
store.append(res)

In [ ]:
# 2016-06-23 - 2017-09-18

res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2016-06-23'), pd.to_datetime('2017-09-18'), ratio=0.85)
store.append(res)

In [ ]:
# 2017-09-18 - 2018-02-06

res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2017-09-18'), pd.to_datetime('2018-02-06'), ratio=0.8)
store.append(res)

In [ ]:
# 2017-09-15 - 2020-04-07

res = plot_and_grab_all_values(df_cdf, pd.to_datetime('2017-09-15'), pd.to_datetime('2020-04-07'), ratio=0.5)
store.append(res)

### Summary of variables

Below a summary for each time period (1 hour duration)

In [ ]:
res = pd.concat(store,axis=1)
res.index = ['23-PT-92504', '45-TT-92606', '45-PT-92608', '23-TIC-92504', 'TT-92604A', 'Calc. Outlet Pressure']
res